In [1]:
# ADD THE LIBRARIES YOU'LL NEED
import os
import sys
from re import M
from typing import List
from collections import Counter
import itertools
from sklearn.decomposition import PCA

import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

from gensim.models import KeyedVectors
from pandas.tseries.offsets import Micro

from sklearn.metrics import accuracy_score, confusion_matrix, \
    f1_score, recall_score, precision_score

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torch.utils.data.dataloader import DataLoader

# import fasttext
import gensim
import torch
import gensim.downloader

In [2]:
global fast_model 
global allwords
print(torch.__version__)
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

1.8.0
cpu


In [3]:
WORD_DICT = None
nltk.download('stopwords')
TOKENIZER = RegexpTokenizer(r'\w+')
NEG_INDICATORS = set(['but', 'if', 'or', 'because', 'until', 'against', 'between',
                      'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some',
                      'such', 'no', 'nor', 'not', 'only', 'so', 'than', 'too', 'very',
                      't', 'can', 'will', 'just', 'don', 'should', "should've", 'now',
                      'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'hasn',
                      'haven', 'isn', 'mightn', 'mustn', 'needn', 'shan', 'shouldn',
                      'wasn', 'weren', 'won', 'wouldn'])
STOPWORDS = set(stopwords.words('english')).difference(NEG_INDICATORS)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vishak/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
def encoded_vectors(text: str):
    if text in GLOVE_MODEL:
        return GLOVE_MODEL[text]
    elif text == "":
        return np.zeros((1, 50))
    else:
        return np.random.randn(1, 50)

In [5]:
def encoded_vectors_fast(text: str):
    global fast_model,allwords
    #print(text)
    if text in fast_model.vocab:
        #ind=allwords.index(text)
        return fast_model[text].reshape((1,100))
    elif text == "":
        return np.zeros((1, 100))
    else:
        return np.random.randn(1, 100)

In [6]:
def encode_data(text: List[str]):
    # This function will be used to encode the reviews using a dictionary(created using corpus vocabulary)

    # Example of encoding :"The food was fabulous but pricey" has a vocabulary of 4 words, each one has to be mapped to an integer like:
    # {'The':1,'food':2,'was':3 'fabulous':4 'but':5 'pricey':6} this vocabulary has to be created for the entire corpus and then be used to
    # encode the words into integers

    # return encoded examples
    vector_list = map(encoded_vectors, text)
    vector_list = map(lambda x: np.reshape(x, (1, 50)), vector_list)
    sent_encoding = np.column_stack(tuple(vector_list))

    # print(sent_encoding)
    # print(sent_encoding.shape)

    return sent_encoding

In [7]:
def encode_data_fast(text: List[str]):
    # This function will be used to encode the reviews using a dictionary(created using corpus vocabulary)

    # Example of encoding :"The food was fabulous but pricey" has a vocabulary of 4 words, each one has to be mapped to an integer like:
    # {'The':1,'food':2,'was':3 'fabulous':4 'but':5 'pricey':6} this vocabulary has to be created for the entire corpus and then be used to
    # encode the words into integers

    # return encoded examples
    #print("FAST")
    vector_list = map(encoded_vectors_fast, text)
    vector_list = map(lambda x: np.reshape(x, (1, 100)), vector_list)
    #print(np.column_stack(tuple(vector_list)).shape)
    sent_encoding = np.expand_dims(np.squeeze(tuple(vector_list)),0)
  # print(sent_encoding.shape)

    # print(sent_encoding)
    # print(sent_encoding.shape)

    return sent_encoding

In [8]:
def create_word_dict(reviews, size: int = 2000):
    corp = list(itertools.chain.from_iterable(list(reviews)))
    counter = Counter(corp)
    most_common_set = counter.most_common()[:min(size, len(counter))]
    print("Most common set", len(most_common_set))
    global WORD_DICT
    WORD_DICT = dict()
    for i, (w, c) in enumerate(most_common_set):
        WORD_DICT[w] = i
    print("word_dict", len(WORD_DICT))

In [9]:
def encode_data2(text: List[str]):
    global WORD_DICT
    if WORD_DICT is None:
        raise ValueError("WORD_DICT not defined")
    result = np.zeros(len(WORD_DICT))
    for w in text:
        if w in WORD_DICT:
            result[WORD_DICT[w]] = 1

    return result

In [10]:
def convert_to_lower(text):
    return str.lower(text)


def remove_punctuation(text):
    # return the reviews after removing punctuations
    pass

In [11]:
def remove_stopwords(text: List[str]):
    # return the reviews after removing the stopwords
    return list(filter(lambda x: x not in STOPWORDS, text))

In [12]:
def perform_tokenization(text):
    # return the reviews after performing tokenization
    return TOKENIZER.tokenize(text)

In [13]:
def perform_padding(data: List[str], pad_len: int = 40):
    # return the reviews after padding the reviews to maximum length
    return data + (pad_len - len(data)) * [""]

In [14]:
def preprocess_data(data: pd.DataFrame, encoder: str = 'glove'):
    # make all the following function calls on your data
    # EXAMPLE:->
    '''
    review = data["reviews"]
    review = convert_to_lower(review)
    review = remove_punctuation(review)
    review = remove_stopwords(review)
    review = perform_tokenization(review)
    review = encode_data(review)
    review = perform_padding(review)
    '''

    reviews = data['reviews']
    reviews = reviews.apply(convert_to_lower)
    reviews = reviews.apply(perform_tokenization)
    # print(reviews.head())
    reviews = reviews.apply(remove_stopwords)
    # print("max len", reviews.apply(len).max())
    if encoder == 'glove':
        reviews = reviews.apply(perform_padding)
        reviews = reviews.apply(encode_data)
    elif encoder == 'bow':
        global WORD_DICT
        if WORD_DICT is None:
            create_word_dict(reviews)
            print("Created Word Dict....")
        reviews = map(encode_data2, list(reviews))
    elif encoder == 'fast':
        reviews = reviews.apply(perform_padding)
        reviews = reviews.apply(encode_data_fast)
    else:
        raise ValueError("Invalid encoder value")
    # reviews =
    # print(reviews.loc[0])
    # print(tuple(reviews)[0])
    reviews = np.row_stack(tuple(reviews))
    # print(reviews.shape)
    # print(reviews.head())

    return reviews


In [15]:
def softmax_activation(x):
    # write your own implementation from scratch and return softmax values(using predefined softmax is prohibited)
    x_exp = torch.exp(x)
    return x_exp / x_exp.sum(dim=1, keepdim=True)

In [17]:
class NeuralNet(nn.Module):

    def __init__(self, reviews, ratings):
        super(NeuralNet, self).__init__()
        self.reviews = torch.from_numpy(reviews.astype(np.float32))
        self.ratings = torch.from_numpy(ratings)
        print("REVIEWS", reviews.shape)
        self.inp_sz = reviews.shape[1]

    def build_nn(self, weighted: bool = False):
        # add the input and output layer here; you can use either tensorflow or pytorch

        #weightSize = 2048
        #self.hidden1 = nn.Linear(self.inp_sz,weightSize)
        #self.hidden2 = nn.Linear(weightSize,weightSize)
        # self.hidden3 = nn.Linear(weightSize,weightSize)
        #self.fc1 = nn.Linear(weightSize, 5)
        #self.lrelu = nn.LeakyReLU()
        '''
        self.rnn = torch.nn.LSTM(input_size = 100, 
                              hidden_size= 200, 
                              num_layers = 3, 
                              batch_first=True)
        '''
        
        self.rnn = torch.nn.GRU(input_size = 100, 
                              hidden_size= 200, 
                              num_layers = 3, 
                              batch_first=True)
                              
        '''
        self.rnn = torch.nn.GRU(input_size = 100, 
                              hidden_size= 200, 
                              num_layers = 3, 
                              batch_first=True,
                              bidirectional=True)
        '''
        self.out_layer = torch.nn.Linear(200, 5)
        self.softmax = torch.nn.LogSoftmax(dim=1)
        if weighted:
            self.loss = nn.CrossEntropyLoss(weight=torch.tensor([4059, 2265, 3612, 6871, 33193.]).reciprocal())#.cuda(0)
        else:
            self.loss = nn.CrossEntropyLoss()

    def train_nn(self, batch_size, epochs):

        use_cuda = torch.cuda.is_available()

        # write the training loop here; you can use either tensorflow or pytorch
        # print validation accuracy
        print("reviews shape", self.reviews.shape)
        print("ratings shape", self.ratings.shape)
        print("reviews shape", self.reviews[:500].shape)
        print("ratings shape", self.ratings[:500].shape)

        train_ds = TensorDataset(self.reviews[:45000], self.ratings[:45000])
        val_ds = TensorDataset(self.reviews[45000:], self.ratings[45000:])
        # train_tensor_ds = train_tensor_ds[:40000]
        print("validation", len(val_ds))
        # print(val_ds[1])
        print("train", len(train_ds))



        train_dl = DataLoader(train_ds, batch_size, shuffle=True)

        if use_cuda : 
          self = self.cuda()
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-4)



        #optim.SGD([
        #        {'params': model.base.parameters()},
         #       {'params': model.classifier.parameters(), 'lr': 1e-3}
          #  ], lr=1e-2, momentum=0.9)

        for epoch in range(epochs):
            print("Epoch:",epoch)
            for x, y  in train_dl:
                if use_cuda : 
                  x,y = x,y
                error = self.loss(self.forward(x), y)
                optimizer.zero_grad()
                error.backward()
                optimizer.step()
            
                #train_ds_0 , val_ds_0 ,train_ds_1, val_ds_1 = train_ds[:][0].cuda() , val_ds[:][0].cuda() , train_ds[:][1].cuda() , val_ds[:][1].cuda()
            #else:
                train_ds_0 , val_ds_0 ,train_ds_1, val_ds_1 = train_ds[:][0] , val_ds[:][0],  train_ds[:][1] , val_ds[:][1]

            #y_train = self.forward(train_ds_0)
            #loss_train = self.loss(y_train, train_ds_1)
            #acc_train = accuracy_score(train_ds[:][1].numpy(), y_train.cpu().argmax(dim=1, keepdim=False).numpy())

            print("validating...")
            y_val = self.forward(val_ds_0)
            loss_val = self.loss(y_val, val_ds_1)
            acc_val = accuracy_score(val_ds[:][1].numpy(), y_val.cpu().argmax(dim=1, keepdim=False).numpy())

            print("Epoch: {} "
                  "ValLoss: {}, ValAcc: {}".format(epoch + 1, loss_val, acc_val))

        print("\nTraining Complete......")

    def forward(self, reviews):
        #out = reviews
        #out = torch.relu(self.hidden1(out))
        #out =  torch.relu(self.hidden2(out))
        # out = F.relu(self.hidden3(out))
        #return softmax_activation((self.fc1(out)))
        #print(reviews.shape)
        out, _ = self.rnn(reviews)
        x = self.out_layer(out[:,-1,:])
        return self.softmax(x)

    def predict(self, reviews):
        reviews_t = torch.from_numpy(reviews.astype(np.float32))
        # return a list containing all the ratings predicted by the trained model
        #self = self.cpu()
        #print
        return self.forward(reviews_t).detach().cpu().numpy()

In [18]:
def encode_label(x: int):
    if x == 1:
        return np.array([[1, 0, 0, 0, 0]])
    if x == 2:
        return np.array([[0, 1, 0, 0, 0]])
    if x == 3:
        return np.array([[0, 0, 1, 0, 0]])
    if x == 4:
        return np.array([[0, 0, 0, 1, 0]])
    if x == 5:
        return np.array([[0, 0, 0, 0, 1]])
    else:
        raise ValueError("UnIdentified Label")

In [19]:
def encode_label_col(labels: pd.DataFrame):
    labels = labels['ratings']
    labels = labels.apply(encode_label)
    labels = np.row_stack(tuple(labels))
    return labels

In [20]:
def evaluate_results(pred_probs: np.ndarray, ground_truth):
    pred_labels = pred_probs.argmax(axis=1)

    print("\nEvaluation Metrics")
    print("Accuracy : ", accuracy_score(ground_truth, pred_labels))
    print("Precision: ", precision_score(ground_truth, pred_labels, average='micro'))
    print("Recall   : ", recall_score(ground_truth, pred_labels, average='micro'))
    print("F1-Score : ", f1_score(ground_truth, pred_labels, average='micro'))
    print("Confusion Matrix")
    conf_df = pd.DataFrame(confusion_matrix(ground_truth, pred_labels),
                           index=["true_1", "true_2", "true_3", "true_4", "true_5"],
                           columns=["pred_1", "pred_2", "pred_3", "pred_4", "pred_5"])
    print(conf_df)

In [21]:
def main(train_file, test_file):
    batch_size, epochs = (16, 20)

    print("11")
    global fast_model
    fast_model = gensim.downloader.load('glove-wiki-gigaword-100')
    '''
    print("22")
    print(type(fast_model_1.wv.vocab))
    print(type(list(fast_model_1.wv.vocab.values())))
   ## PCA fast_model = 
    allwordvec=[]
    global allwords
    allwords=list(fast_model_1.wv.vocab.keys())
    for word in allwords:
      allwordvec.append(fast_model_1[word])
    #pca = PCA(n_components=50, svd_solver='full')
    print(len(allwordvec))
    print(len(allwordvec[0]))
    pca.fit(np.asarray(allwordvec).transpose())
    fast_model = pca.components_.transpose()
    print(fast_model.shape)
    fast_model_1 = 0
    '''

     
    # Load training dataframe
    #train_data: pd.DataFrame = pd.read_csv(train_file, header=0, index_col=0)
    #train_data = train_data.sample(frac=1, random_state=0).reset_index(drop=True)
    test_data: pd.DataFrame = pd.read_csv(test_file, header=0, index_col=0)

    #print("Train Data distribution")
    #print(train_data.groupby(['ratings']).agg(['count']))
    print("Test Data distribution")
    print(test_data.groupby(['ratings']).agg(['count']))

    #print('Train Dataframe....')
    #print(train_data.head(), "\n")
    print('Test Dataframe....')
    print(test_data.head(), "\n")

    # Embedding Supported: fast , glove , bow
    #train_reviews = preprocess_data(train_data, encoder='fast')  # "gov"
    test_reviews = preprocess_data(test_data, encoder='fast')
    print("090")

    #train_ratings = train_data['ratings'].to_numpy().reshape((-1,)) - 1
    test_ratings = test_data['ratings'].to_numpy().reshape((-1,)) - 1

    print("123")
    
    '''
    model = NeuralNet(train_reviews, train_ratings)
    #model=model.cuda()
    print("456")
    model.build_nn(weighted=True)
    print("789")
    model.train_nn(batch_size, epochs)
    print("101112")
    '''
    
    
    model = torch.load('saved_model')
    #torch.save(model, "saved_model")
    pred_probs: np.ndarray = model.predict(test_reviews)

    
    evaluate_results(pred_probs, test_ratings)

    prediction_df = pd.DataFrame(pred_probs,
                                 columns=["prob_1", "prob_2", "prob_3", "prob_4", "prob_5"])
    prediction_df['predicted_Class'] = pred_probs.argmax(axis=1) + 1
    # prediction_df["true_ratings"] = test_ratings +1
    print("\nPredictions\n")
    print(prediction_df)
    return prediction_df, pred_probs




In [ ]:
predictions, _ = main("train.csv","gold_test.csv")

In [42]:
import tkinter as tk
root=tk.Tk()
 
# setting the windows size
root.geometry("600x400")

# declaring string variable
# for storing name and password
name_var=tk.StringVar()
#passw_var=tk.StringVar()
rating_var=tk.StringVar()
global sentiment_test
global rating_test

# defining a function that will
# get the name and password and
# print them on the screen
def submit(rating_entry):
    
    global sentiment
    global rating
 
    name=name_var.get()
    #password=passw_var.get()
     
    print("The name is : " + name)
    #print("The password is : " + password)
    
    sentiment_test = name
    #rating_test = password
    rating = fn(name)
    rating_entry.insert(0, rating)

    #name_var.set("")
    #passw_var.set("")
    
# creating a label for
# name using widget Label
name_label = tk.Label(root, text = 'Sentiment', font=('calibre',10, 'bold'))
# creating a entry for input
# name using widget Entry
name_entry = tk.Entry(root,textvariable = name_var, font=('calibre',10,'normal'))

# creating a label for password
#passw_label = tk.Label(root, text = 'Dummy rating', font = ('calibre',10,'bold'))
  
# creating a entry for password
#passw_entry=tk.Entry(root, textvariable = passw_var, font = ('calibre',10,'normal'))


# creating a label for password
rating_label = tk.Label(root, text = 'Rating', font = ('calibre',10,'bold'))
  
# creating a entry for password
rating_entry=tk.Entry(root, textvariable = rating_var, font = ('calibre',10,'normal'))



# creating a button using the widget
# Button that will call the submit function
sub_btn=tk.Button(root,text = 'Submit', command = lambda: submit(rating_entry))

# placing the label and entry in
# the required position using grid
# method
name_label.grid(row=0,column=0)
name_entry.grid(row=0,column=1)
#passw_label.grid(row=1,column=0)
#passw_entry.grid(row=1,column=1)
rating_label.grid(row=2,column=0)
rating_entry.grid(row=2,column=1)
sub_btn.grid(row=5,column=1)

  
# performing an infinite loop
# for the window to display
root.mainloop()

The name is : This movie is awful
This movie is awful
11
Test Data distribution
        reviews
          count
ratings        
1          1271
2           630
3           913
4          1404
5          5784
Test Dataframe....
                                             reviews  ratings
0  Doesn't work at ALL. Don't waste your money or...        1
1  What crap.  Would need a lot more power to do ...        1
2  Has no suction and didn't work. Not worth trying.        1
3  That is definitely a trash. Unable to clean an...        1
4  Didn't even worked on cleaning the ears at all...        1 

090
123

Evaluation Metrics
Accuracy :  0.6261747650469907
Precision:  0.6261747650469907
Recall   :  0.6261747650469907
F1-Score :  0.6261747650469907
Confusion Matrix
        pred_1  pred_2  pred_3  pred_4  pred_5
true_1     781     309     111      20      50
true_2     187     228     145      39      31
true_3     101     207     413     134      58
true_4      34      96     351     516    

In [40]:
import csv
def fn(sentiment):
    print(sentiment)
    with open('trial.csv', 'a', newline='') as file:
        writer = csv.writer(file)
    
        writer.writerow([1, sentiment, "3"])
    
    predictions, _ = main("train.csv","trial.csv")
    return(predictions['predicted_Class'].to_list()[-1])